<a href="https://colab.research.google.com/github/stheria4/sds510/blob/master/Module5Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Naive Bayes Classifier for Jeopardy Questions

**Name:** Sean Theriault
**Student ID:** stheria4
**Course:** SDS 510 – Python for Data Wrangling  
**Date:** 11/19/2025
**Project:** Module 5 - Basics Badge

This script imports the necessary libraries.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

Load Dataset from sharable google drive link

In [19]:
# Load Dataset
url = "https://drive.google.com/uc?export=download&id=1DCMBbMHAtGNnAZi2H8iupc429HSlDhgZ"
df = pd.read_json(url)

### Simple Jeopardy Classifier

Basic setup: convert values, make high/low labels, turn the text into bag-of-words, and train a Naive Bayes model. I added more comments in this section based on the feedback from my last submission.

In [20]:
# Turn the "value" field into numbers
# I kept getting errors turning the money values into numbers. ChatGPT suggested
# stripping out "$" and commas and using try/except, which fixed it.
def to_num(v):
    if isinstance(v, str):
        v = v.replace("$", "").replace(",", "")
        try:
            return int(v)
        except:
            return None
    return None

df["ValueNum"] = df["value"].apply(to_num)
df = df.dropna(subset=["ValueNum"])

# High/low label, using 1000 as cutoff
# I'm not totally sure if 1000 is the best cutoff, but it seemed like a reasonable
# split for Jeopardy values, and keeps things simple.
df["Label"] = df["ValueNum"].apply(lambda x: 1 if x >= 1000 else 0)

# Use the raw question text
# I originally tried doing more cleaning, but it kept messing things up,
# so I just decided to use the plain text. The vectorizer still seems to work.
texts = df["question"].fillna("").astype(str)

# Bag-of-words word vectors
vec = CountVectorizer()
X = vec.fit_transform(texts)
y = df["Label"]

# Train/test split
# I used 20% for testing since that's what most examples online use.
# I'm still learning how this part works exactly.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

# Train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# Evaluate
pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)
print("Accuracy:", acc)

# Save output
with open("classifier_output.txt", "w") as f:
    f.write("Accuracy: " + str(acc))

# The accuracy isn’t very high, but I honestly ran out of time. This basic classifier
# took me longer than expected to get working, so I just went with the simplest version.

print("Saved classifier_output.txt")

Accuracy: 0.690084388185654
Saved classifier_output.txt
